In [2]:
import IPython
print(IPython.__version__)  # Check your IPython version

%load_ext autoreload
%autoreload 2

8.18.1


In [3]:
%reload_ext autoreload

In [4]:
import torch
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer, AutoConfig
import json
from typing import List, Optional, Tuple, Dict
import time
from dataclasses import dataclass
import lm_eval
from lm_eval import evaluator, tasks, utils
from lm_eval.api.model import LM
from lm_eval.api.registry import register_model
from lm_eval.tasks import get_task_dict
from functools import partial
from tqdm import tqdm
import numpy as np
import multiprocessing
import ftfy
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from itertools import zip_longest

print(f"torch version: {torch.__version__}")
print(f"transformers version: {transformers.__version__}")

/opt/anaconda3/envs/deep-learning-env/lib/python3.9/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/deep-learning-env/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <4C793A59-B32A-3AF1-BEA5-03AD7C5C80C6> /opt/anaconda3/envs/deep-learning-env/lib/python3.9/site-packages/torchvision/image.so
  Reason: tried: '/opt/anaconda3/envs/deep-learning-env/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/deep-learning-env/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/deep-learning-env/lib/python3.9/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/deep-learning-env/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there 

torch version: 2.6.0.dev20241112
transformers version: 4.46.3


In [5]:
import sys, os
sys.path.append('../')

In [7]:
from data.config import BenchmarkConfig, DatasetConfig
from data.data import DatasetManager, ModelDataset
from data.metrics import BenchmarkMetrics
from models.h2o.h2o_gptneox import GPTNeoXAttention_Mask, convert_kvcache_gpt_neox_heavy_recent
from models.h2o.h2o_llama import LlamaAttention_heavy_hitter, convert_kvcache_llama_heavy_recent
from models.h2o.h2o_opt import OPTAttention_Mask, convert_kvcache_opt_heavy_recent
from models.base_models import ModelLoader
from tests.run_benchmark import run_benchmark


In [6]:
device = (
    "mps" if torch.backends.mps.is_available() else
    "cuda" if torch.cuda.is_available() else
    "cpu"
)
print(f"Using device: {device}")

Using device: mps


In [7]:
# Mini benchmark configuration
mini_config = BenchmarkConfig(
    # Use a small model that works well on Mac
    model_name="facebook/opt-125m",  # Tiny model for testing
    model_type="opt",
    
    # Minimal dataset samples
    datasets=[
        DatasetConfig(name="hellaswag", splits=["validation[:10]"]),  # Only 10 examples
        DatasetConfig(name="piqa", splits=["validation[:10]"])        # Only 10 examples
    ],
    
    # Small batch and sequence settings
    eval_batch_size=2,
    inference_batch_size=1,
    sequence_length=128,  # Shorter sequences
    
    # Minimal benchmark settings
    measure_perplexity=True,
    measure_latency=True,
    measure_throughput=False,  # Skip for quick test
    measure_kv_cache=False,    # Skip for quick test
    num_runs=1,               # Single run
    
    # Hardware settings
    dtype="float16",
    device=device,            # Use detected device
    gradient_checkpointing=False
)

In [ ]:
full_results = run_benchmark(full_test_config)

In [8]:
# Full dataset test config
full_test_config = BenchmarkConfig(
    # Model settings - keeping small model for Mac testing
    model_name="facebook/opt-125m",
    model_type="opt",
    
    # All datasets from the original file
    datasets=[
        # Working datasets - no changes needed
        DatasetConfig(name="piqa", splits=["validation[:100]"]),
        DatasetConfig(name="hellaswag", splits=["validation[:100]"]),
        DatasetConfig(name="EleutherAI/lambada_openai", splits=["test[:100]"]),
        
        # Fixed configurations for datasets that need them
        DatasetConfig(name="winogrande", splits=["validation[:100]"], config="winogrande_xl"),
        DatasetConfig(name="pubmed_qa", splits=["train[:100]"], config="pqa_labeled"),
        
        # SuperGLUE datasets need to be loaded differently
        DatasetConfig(name="super_glue", splits=["validation[:100]"], config="boolq"),
        DatasetConfig(name="super_glue", splits=["validation[:100]"], config="cb"),
        DatasetConfig(name="super_glue", splits=["validation[:100]"], config="copa"),
        DatasetConfig(name="super_glue", splits=["validation[:100]"], config="multirc"),
        DatasetConfig(name="super_glue", splits=["validation[:100]"], config="record"),
        DatasetConfig(name="super_glue", splits=["validation[:100]"], config="wic"),
        DatasetConfig(name="super_glue", splits=["validation[:100]"], config="wsc.fixed"),
    ],
    
    # Mac-friendly settings
    eval_batch_size=4,
    inference_batch_size=1,
    sequence_length=256,
    
    # All metrics enabled
    measure_perplexity=True,
    measure_latency=True,
    measure_throughput=True,
    measure_kv_cache=True,
    num_runs=1,
    
    # Mac settings
    dtype="float16",
    device="mps",
    gradient_checkpointing=False
)

In [11]:
test_config = BenchmarkConfig(
    # Required model settings
    model_name="meta-llama/Llama-3.2-3B-Instruct",
    model_type="llama",
    
    # Memory optimizations
    dtype="float16",
    eval_batch_size=2,      # Can try batch size 2 since it's smaller
    inference_batch_size=1,
    sequence_length=512,    # Reduced sequence length
    gradient_checkpointing=True,
    
    # Reduced dataset list to save memory
    datasets=[
        DatasetConfig(name="piqa", splits=["validation[:100]"]),
        DatasetConfig(name="hellaswag", splits=["validation[:100]"]),
        DatasetConfig(name="EleutherAI/lambada_openai", splits=["test[:100]"]),
    ],
    
    # Hardware settings
    device="mps",
    
    # Keep default values for other parameters
    attention_type="default",
    heavy_ratio=0.1,
    recent_ratio=0.1,
    num_fewshot=0,
    shrink=False,
    min_seq=None,
    measure_perplexity=True,
    measure_latency=True,
    measure_throughput=True,
    measure_kv_cache=True,
    num_runs=3
)

In [13]:
test_results = run_benchmark(test_config)


Initializing components...


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Perplexity: 2711.75


Measuring latency (batch_size=1): 100%|█| 50/50 [01:13<00:00,  1.48s/batch, avg_


Average latency: 897.69ms
P90 latency: 900.96ms
Throughput: 104.45 tokens/sec


Measuring KV cache size: 254param [00:00, 3589.45param/s, cache_size=672.00MB]

KV Cache size: 672.00MB

Results saved to benchmark_results_Llama-3.2-3B-Instruct_20241202_173303.json


In [10]:
from tests.run_benchmark import run_cs_helm_benchmark

In [11]:
# HELM benchmark configuration
helm_config = BenchmarkConfig(
    # Model settings
    model_name="meta-llama/Llama-2-7b-hf",  # Or your preferred model
    model_type="llama",
    
    # Memory optimizations for HELM
    dtype="float16",
    eval_batch_size=4,      # Smaller batch size for HELM evaluation
    inference_batch_size=1,  # Keep at 1 for accurate latency measurements
    sequence_length=512,     # Reduced sequence length
    gradient_checkpointing=True,
    
    # HELM-specific dataset configurations
    datasets=[
        # Core HELM scenarios
        DatasetConfig(name="mmlu", splits=["validation[:100]"], config="humanities"),
        DatasetConfig(name="truthfulqa", splits=["validation[:100]"], config="mc"),
        DatasetConfig(name="hellaswag", splits=["validation[:100]"]),
        DatasetConfig(name="winogrande", splits=["validation[:100]"], config="winogrande_xl"),
        DatasetConfig(name="bbq", splits=["validation[:100]"], config="all"),
    ],
    
    # Hardware settings
    device="mps",  # Adjust based on your hardware
    
    # Benchmark settings
    measure_perplexity=True,
    measure_latency=True,
    measure_throughput=True,
    measure_kv_cache=True,
    num_runs=1  # Increase for more stable results
)

In [25]:
config = BenchmarkConfig(
    model_name="meta-llama/Llama-2-7b",
    model_type="llama"
)

In [26]:
config.switch_dataset("mmlu")  # Will evaluate using all the metrics above

In [28]:
results = run_helm_benchmark(config)

Local files do not exist for HuggingFace tokenizer: meta-llama/Llama-2-7b. Downloading...


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


ValueError: Converting from Tiktoken failed, if a converter for SentencePiece is available, provide a model path with a SentencePiece tokenizer.model file.Currently available slow->fast convertors: ['AlbertTokenizer', 'BartTokenizer', 'BarthezTokenizer', 'BertTokenizer', 'BigBirdTokenizer', 'BlenderbotTokenizer', 'CamembertTokenizer', 'CLIPTokenizer', 'CodeGenTokenizer', 'ConvBertTokenizer', 'DebertaTokenizer', 'DebertaV2Tokenizer', 'DistilBertTokenizer', 'DPRReaderTokenizer', 'DPRQuestionEncoderTokenizer', 'DPRContextEncoderTokenizer', 'ElectraTokenizer', 'FNetTokenizer', 'FunnelTokenizer', 'GPT2Tokenizer', 'HerbertTokenizer', 'LayoutLMTokenizer', 'LayoutLMv2Tokenizer', 'LayoutLMv3Tokenizer', 'LayoutXLMTokenizer', 'LongformerTokenizer', 'LEDTokenizer', 'LxmertTokenizer', 'MarkupLMTokenizer', 'MBartTokenizer', 'MBart50Tokenizer', 'MPNetTokenizer', 'MobileBertTokenizer', 'MvpTokenizer', 'NllbTokenizer', 'OpenAIGPTTokenizer', 'PegasusTokenizer', 'Qwen2Tokenizer', 'RealmTokenizer', 'Refor

In [29]:
!pip install crfm-helm
!pip install transformers
!pip install torch

# Clone your repository (assuming it's on GitHub)
!git clone https://github.com/thomasjhuang/liquid-fusion.git
%cd liquid-fusion

# Create necessary directories
!mkdir -p helm_env benchmark_results

Cloning into 'liquid-fusion'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 29 (delta 3), reused 29 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (29/29), 14.84 KiB | 14.84 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/Users/thhuang/llm-training/liquid-fusion/scripts/liquid-fusion


In [31]:
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Model: {torch.cuda.get_device_name(0)}")
    print(f"Memory Available: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

GPU Available: False
